In [19]:
def load_hero_names():
    hero_dict = dict()
    
    with open('../dataset/Marvel-Names.txt',encoding='ISO-8859-1') as f:
        for line in f:
            tok = line.split()[:2]
            hero_dict[int(tok[0])] = tok[1].replace('"','')
            
    return hero_dict

In [23]:
from pyspark import SparkConf, SparkContext
import collections

conf = SparkConf().setMaster("local").setAppName("RatingsHistogram")
sc = SparkContext.getOrCreate(conf=conf)

hero_dict = sc.broadcast(load_hero_names())
# 얘를 필요한 executor에게 보내는 방식

lines = sc.textFile("file:/home/ej/github/codingplace/spark/dataset/Marvel-Graph.txt")


In [55]:
hero_count = lines.flatMap(lambda x: x.split()).map(
    lambda x: (int(x), 1)).reduceByKey(lambda x, y: x+y).map(lambda x:(x[1],x[0]))

In [61]:
sorted_count = hero_count.sortByKey().map(
    lambda x: (hero_dict.value[x[1]], x[0]))

In [63]:
sorted_count.max()

('ÁGAMORA', 243)

In [62]:
sorted_count.collect()[::-1][:5]

[('CAPTAIN', 1937),
 ('SPIDER-MAN/PETER', 1745),
 ('IRON', 1532),
 ('THING/BENJAMIN', 1429),
 ('WOLVERINE/LOGAN', 1397)]